# BitChute Scraper Examples

In [1]:
import bitchute as bc
api = bc.BitChuteAPI(max_workers=3)
df = api.get_trending_videos('day', limit=5, include_details=True, parallel_details=True)
df_with_thumbs = api.download_thumbnails(df, "test_thumbs/")
print(f"Success: {len(df_with_thumbs)} videos, thumbnails: {'local_thumbnail_path' in df_with_thumbs.columns}")

NameError: name 'selection_map' is not defined

In [ ]:
import bitchute as bc

In [ ]:
# Initialize the BitChute API client
api = bc.BitChuteAPI(verbose=False)

## Get Platform Recommendations

In [ ]:
# Trending Videos
trending = api.get_trending_videos('day', limit=3, include_details=True) # Other options: 'week', 'month'. Defaults to 'day'.
print(f"Success! Got {len(trending)} videos")
print()
trending.head(2)

In [ ]:
# Popular Videos (now named "Fresh" on the frontend)
popular = api.get_popular_videos(limit=500) # You can specify a limit, default is 100.
print(f"Success! Got {len(popular)} videos")
print()
popular.head(2)

In [ ]:
# All Videos
all_videos = api.get_all_videos()
print(f"Success! Got {len(all_videos)} videos")
print()
all_videos.head(2)

In [ ]:
# Trending Hashtags
hashtags = api.get_trending_hashtags(limit=500)
print(f"Success! Got {len(hashtags)} hashtags")
print()
hashtags.head(2)

## Search the BitChute Platform

In [ ]:
# Search Videos

search = api.search_videos('bitcoin', limit=50,sensitivity='normal', sort='new') 
# sensitivity alternatives : 'nsfw', 'nsfl' defaults to 'normal'
# sort alternatives: 'views', 'old' defaults to 'new'
print(f"Success! Got {len(search)} videos")
print()
search.head(2)

In [ ]:
# Search Channels

channels = api.search_channels('truth', limit=50, sensitivity='normal') 
# sensitivity alternatives : 'nsfw', 'nsfl' defaults to 'normal'
print(f"Success! Got {len(channels)} channels")
print()
channels.head(2)

In [ ]:
# Get channel information:
# Get channel details
channel = api.get_channel_details('R7juPfa5uBpC')
print(f"Channel: {channel.name}")
print(f"Videos: {channel.video_count}, Views: {channel.view_count}")
print(f"Last upload: {channel.last_video_published}")

channel.__dict__

In [ ]:
# Get channel videos
videos = api.get_channel_videos('R7juPfa5uBpC', limit=100)
videos

In [ ]:
# Updated bitchute_examples.py - Show new features

import bitchute as bc
import time
from pathlib import Path

# Initialize API with parallel processing enabled
api = bc.BitChuteAPI(verbose=True, max_workers=5)

print("🚀 BitChute API Examples - Enhanced Version\n")

# Example 1: Basic trending videos (fast)
print("1. Basic Trending Videos")
trending = api.get_trending_videos('day', limit=20)
print(f"   Retrieved {len(trending)} videos")
print(f"   Columns: {list(trending.columns)[:5]}...")
print()

# Example 2: Trending with parallel detail enrichment
print("2. Trending Videos with Parallel Detail Enrichment")
start_time = time.time()
detailed_trending = api.get_trending_videos(
    'day', 
    limit=10, 
    include_details=True,
    parallel_details=True  # New feature
)
elapsed_time = time.time() - start_time

print(f"   Retrieved {len(detailed_trending)} videos with details")
print(f"   Time taken: {elapsed_time:.2f} seconds")
print(f"   Has like counts: {'like_count' in detailed_trending.columns}")
print()

# Example 3: Download thumbnails with deduplication
print("3. Download Thumbnails")
if len(trending) > 0:
    # Create thumbnails directory
    thumb_dir = Path("examples_thumbnails")
    thumb_dir.mkdir(exist_ok=True)
    
    # Download thumbnails for first 5 videos
    sample_videos = trending.head(5)
    
    start_time = time.time()
    videos_with_thumbs = api.download_thumbnails(
        sample_videos,
        download_path=str(thumb_dir),
        check_existing=True,
        add_path_column=True
    )
    elapsed_time = time.time() - start_time
    
    print(f"   Downloaded thumbnails for {len(videos_with_thumbs)} videos")
    print(f"   Time taken: {elapsed_time:.2f} seconds")
    print(f"   Has thumbnail paths: {'local_thumbnail_path' in videos_with_thumbs.columns}")
    
    # Show downloaded files
    downloaded = videos_with_thumbs[videos_with_thumbs['local_thumbnail_path'] != '']
    print(f"   Successfully downloaded: {len(downloaded)} files")
    for _, row in downloaded.iterrows():
        file_path = Path(row['local_thumbnail_path'])
        if file_path.exists():
            file_size = file_path.stat().st_size / 1024  # KB
            print(f"     {file_path.name}: {file_size:.1f} KB")
print()

# Example 4: Large dataset with all features
print("4. Large Dataset Example")
start_time = time.time()

# Get more videos
all_videos = api.get_all_videos(limit=100)
print(f"   Retrieved {len(all_videos)} videos")

# Analyze the data
from bitchute.utils import DataAnalyzer
analyzer = DataAnalyzer()
analysis = analyzer.analyze_videos(all_videos)

print(f"   Analysis results:")
if 'views' in analysis:
    print(f"     Total views: {analysis['views']['total']:,}")
    print(f"     Average views: {analysis['views']['average']:,.0f}")
    print(f"     Top channel: {list(analysis.get('top_channels', {}).keys())[0] if analysis.get('top_channels') else 'N/A'}")

elapsed_time = time.time() - start_time
print(f"   Total time: {elapsed_time:.2f} seconds")
print()

# Example 5: Export data in multiple formats
print("5. Data Export")
from bitchute.utils import DataExporter

exporter = DataExporter()
exported_files = exporter.export_data(
    all_videos.head(50),  # Export first 50 for demo
    "bitchute_sample",
    formats=['csv', 'json']
)

for format_name, filepath in exported_files.items():
    file_size = Path(filepath).stat().st_size / 1024  # KB
    print(f"   Exported {format_name.upper()}: {filepath} ({file_size:.1f} KB)")
print()

# Example 6: Video search with details
print("6. Video Search with Parallel Details")
search_results = api.search_videos(
    'climate change',
    limit=10,
    include_details=True,
    parallel_details=True
)

print(f"   Found {len(search_results)} videos for 'climate change'")
if len(search_results) > 0:
    top_result = search_results.iloc[0]
    print(f"   Top result: {top_result['title'][:50]}...")
    print(f"   Views: {top_result.get('view_count', 'N/A')}")
    print(f"   Channel: {top_result.get('channel_name', 'N/A')}")
print()

# Example 7: API Statistics
print("7. API Performance Statistics")
stats = api.get_api_stats()
print(f"   Requests made: {stats['requests_made']}")
print(f"   Error rate: {stats['error_rate']:.2%}")
print(f"   Has cached token: {stats['token_cached']}")
print()

# Performance comparison example
print("8. Performance Comparison: Sequential vs Parallel")

# Sequential enrichment
start_time = time.time()
sequential_videos = api.get_trending_videos(
    'week', 
    limit=5, 
    include_details=True,
    parallel_details=False
)
sequential_time = time.time() - start_time

# Parallel enrichment  
start_time = time.time()
parallel_videos = api.get_trending_videos(
    'week', 
    limit=5, 
    include_details=True,
    parallel_details=True
)
parallel_time = time.time() - start_time

print(f"   Sequential (5 videos): {sequential_time:.2f}s")
print(f"   Parallel (5 videos): {parallel_time:.2f}s")
print(f"   Speedup: {sequential_time/parallel_time:.1f}x faster")
print()

print("✅ All examples completed!")

# Cleanup
api.cleanup()